#利用LLM从医疗文本中提取FHIR数据

##原理：

准备：使用[中文医疗信息处理评测基准CBLUE](https://tianchi.aliyun.com/dataset/95414?lang=zh-cn)中的临床发现事件抽取（CHIP-CDEE）数据集，对阿里[Qwen2-7B-Instruct](https://huggingface.co/Qwen/Qwen2-7B-Instruct)大模型进行监督微调（SFT），获得针对临床发现事件抽取任务特化的大模型[Qwen2-7B-Instruction-extraction](https://huggingface.co/neonicholasi/Qwen2-7B-Instruction-extraction)，用于支撑后续的医疗文本处理任务。

整个文本处理过程分两步

###第一步：

利用Qwen2-7B-Instruction-extraction从医疗文本中以CHIP-CDEE要求的格式提取临床发现事件（主要包含临床观察、诊断），临床发现事件以如下格式JSON呈现：


```json
text: 表示病历或者医学影像报告
    event: 列表结构，由一个或者多个事件四元组组成
        core_name: 主体词，字符串
        tendency: 发生状态，字符串，如果没有发生状态，默认为""
        character: 描述词，列表结构，如果没有描述词，默认为[]
        anatomy_list: 解剖部位，列表结构，如果没有解剖部位，默认为[]

```

###第二步:

以第一步中获得的临床发现事件为基础，再次调用Qwen2-7B-Instruction-extraction，将临床发现事件转换为FHIR R4版本的资源，并以JSON数组包装多个FHIR资源。
尽管Qwen2具备一定的FHIR知识，但经测试，此模型并未针对该任务进行过微调，因此Qwen2-7B-Instruction-extraction在转换FHIR资源时可观测到如下限制：
1. 有FHIR资源实例幻觉，如对同一临床发现重复生成多个资源
2. 有FHIR资源属性幻觉，生成一些未被FHIR R4定义的属性
3. 输出不完整，由于FHIR资源属性远较第一步中形成的事件属性丰富，生成的FHIR资源报文长度远大于原文本长度和临床事件数组的报文长度。因此该模型标配的最大2048字节的文本生成长度（max_new_tokens）常不足以容纳完整的FHIR资源数组所需的长度，造成生成中断，产生不完整的FHIR资源数组。

##为流畅演示，使用前设置如下：

1. 启动[大模型服务](https://colab.research.google.com/drive/1nGT1IZ_1KyIwQfaBG9FgGxRGYpK08QqQ#scrollTo=kIt2Ma2WGAQC)
2. 设置下文的appURL为服务地址
3. 启动[IRIS Production](http://localhost:52773/csp/healthshare/fhirserver/EnsPortal.ProductionConfig.zen?$NAMESPACE=FHIRSERVER&$NAMESPACE=FHIRSERVER&)，并将BO TextHandler.operations.AIOperation的连接地址更新为服务地址
4. 因跨容器通信，需将intServiceUrl中的ip部分改为容器iris-showcase分配到的IP地址
5. 拷贝一段临床文本以备输入

In [11]:
appURL = 'http://' + '2a69-34-125-113-171.ngrok-free.app'
extractService = '/extract'
fhirService = '/fhir'
intServiceUrl = 'http://'+'172.23.0.3'+':52773/AIFunc/Text2FHIR'

##医疗文本转临床发现事件

医疗本文样本：

入院前1+年，患儿无明显诱因开始出现阴茎增长，伴生长加速，无变声，无胡须，无痤疮，无皮肤色素沉着，在我院门诊诊断“先天性肾上腺皮质增生症”后规律予以激素替代治疗至今，期间规律于我院随访，现口服氢化可的松 10mg，3/日。

患儿精神睡眠可，大小便未见明显异常。

无声音嘶哑、吞咽困难，在某医院输液治疗无缓解。

2年前患者无明显诱因出现腹胀、腹围增加，腹胀与进食无明显关系，轻度乏力，能完成日常生活及工作，无腹痛、腹泻，无呕血、黑便及发热等，2015年5月9日就诊于我院，查甲胎蛋白增高（具体不详），腹部影像学检查诊断为“原发性肝癌”，于2015年5月15日在我院行tace术，口服恩替卡韦（0.5mg qd）抗病毒治疗，保肝等对症治疗好转出院。

患者发病以来，神志 清晰，精神可，胃纳可，大便如常、大便次数增多，小便如常，体重下降4kg。

3月前开始出现便血，量不详，便血与粪便混合，于2016-02-15于某医院行肠镜提示直肠中分化腺癌，结肠多发息肉。

21天前，患儿因“诊断右股骨骨肉瘤3期2月余，拟行第3次化疗”入院，予以化疗方案（甲氨蝶呤d1，长春新碱d1)化疗，复查血常规基本正常后出院,不伴发热、不伴腹痛，今日为行hifu手术治疗入院。

患者于入院前3个月（2014.06.25）因“右乳癌”于我院在全麻下行右腋窝前哨淋巴结探查活检+右乳癌改良根治术，手术顺利，术后诊断：（右）乳浸润性导管癌，腋窝淋巴结未见癌转移（0/9），免疫组化示er（+++）70%pr（－）cerbb-2（－） ki-67（+）20%。

入院前1+月，患者因“大便习惯改变2+月，伴便血15天”于我科住院治疗，完善相关辅助检查回示：病理检查：直肠腺癌；下腹部：乙状结肠、直肠肠壁不均匀性增厚，考虑肿瘤性病变。

In [9]:
#输入医疗文本，可从样本中拷贝
clinicText = input("请输入医疗文本: ")

In [ ]:
import requests
import json

# 目标API的URL
url = appURL + extractService

# POST请求的数据，这里以JSON格式为例
data = {
    "text": clinicText
}

# 设置请求头，特别指明Content-Type为application/json
headers = {'Content-Type': 'application/json'}

# 发起POST请求
response = requests.post(url, data=json.dumps(data), headers=headers)

# 检查请求是否成功
if response.status_code == 200:
    # 解析并打印返回的JSON数据
    entity_data = response.json()
    print("临床发现事件数据:")
    print(json.dumps(entity_data, ensure_ascii=False, indent=4))  # 使用indent参数让输出更加易读
else:
    print(f"请求失败，状态码：{response.status_code}")

##临床发现事件转FHIR

In [ ]:
import requests
import json
# 目标API的URL
url = appURL + fhirService

# POST请求的数据，这里以JSON格式为例
data = {
    "text": json.dumps(entity_data, ensure_ascii=False, indent=4)
}

# 设置请求头，特别指明Content-Type为application/json
headers = {'Content-Type': 'application/json'}

# 发起POST请求
response = requests.post(url, data=json.dumps(data), headers=headers)

# 检查请求是否成功
if response.status_code == 200:
    # 解析并打印返回的JSON数据
    fhir_data = response.json()
    print("FHIR资源数据:")
    print(json.dumps(fhir_data, ensure_ascii=False, indent=4))  # 使用indent参数让输出更加易读
else:
    print(f"请求失败，状态码：{response.status_code}")

##通过IRIS集成大模型能力，从临床文本中提取FHIR资源 （2～3分钟）
[Production](http://localhost:52773/csp/healthshare/fhirserver/EnsPortal.ProductionConfig.zen?$NAMESPACE=FHIRSERVER&$NAMESPACE=FHIRSERVER&)

In [ ]:
import requests
import base64
import requests
import json
# 目标API的URL
url = intServiceUrl

# POST请求的数据，这里以JSON格式为例
data = {
    "text": clinicText
}

# 设置你的用户名和密码
username = 'superuser'
password = 'SYS'

# 将用户名和密码编码为Base64字符串，用于Authorization头
credentials = f"{username}:{password}"
encoded_credentials = base64.b64encode(credentials.encode()).decode('utf-8')

# 设置请求头，特别指明Content-Type为application/json
# 构建请求头，包括Authorization
headers = {
    "Authorization": f"Basic {encoded_credentials}",
    "Content-Type": "application/json"  # 假设API需要JSON格式的内容类型
}

# 发起POST请求
response = requests.post(url, json=data, headers=headers)

# 检查请求是否成功
if response.status_code == 200:
    # 解析并打印返回的JSON数据
    fhir_data = response.json()
    print("FHIR资源数据:")
    print(json.dumps(fhir_data, indent=4))  # 使用indent参数让输出更加易读
else:
    print(f"请求失败，状态码：{response.status_code}")